# 여행 플래너 Chat Bot 구현

## 1. Setting

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## 2. Prompt Engineering

- Input : 일정(날짜, 일수 등), 여행 장소 
- Output : 여행 기간 동안 장소별 세부 계획
- Format :

| 날짜 및 시간 | 장소 | 세부 계획 |
|---|---|---|
| Time 1 | 장소 A | - |
| Time 2 | 장소 B | - |
| Time 3 | 장소 C | - |
| Time 4 | 장소 D | - |


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

summarize_templete = """
{text}

위에 입력된 텍스트를 다음 항목으로 요약해주세요: 
- 여행 일정 :
- 교통편 일정 :
- 여행 장소 :
- 여행 스타일 :
- 예산 :
- 추천 숙소 :"""

summarize_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 일정 작성을 도와주는 AI 어시스턴트입니다."),
    ("human", summarize_templete)
])

planner_prompt = ChatPromptTemplate.from_template("""
다음 텍스트의 여행 일정을 기반으로 세부 여행 일정을 짜주세요.
텍스트: {summary}
규칙:
1. 날짜 및 시간과 장소, 세부 계획 항목으로 표 형태로 작성하세요.
2. 여행 스타일과 추천 숙소, 예산에 맞추어 동선을 고려하여 장소를 추천하세요.
답변:""")

## 3. Chain Testing

In [ ]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap, RunnableLambda

model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.4,
    top_p=0.7
)

# 체인 구성
summarize_chain = summarize_prompt | model
planner_chain = planner_prompt | model | StrOutputParser()

# 최종 체인
chain = (
    summarize_chain |
    RunnableLambda(lambda x: {"summary": x.content}) |
    RunnableMap({
        "summary": itemgetter("summary"),
        "plan": planner_chain
    }) |
    RunnableLambda(lambda x: f"<요약>\n{x['summary']}\n\n<일정>\n{x['plan']}")
)

In [4]:
text = """내일 오전 8시에 서울역에서 출발해서 오전 11시에 부산역에 도착해.
2박 3일동안 부산 기장군 부근에서 여행하고 싶어.
맛있는 거 먹으면서 돌아다니고 싶고, 명소도 가고 싶어.
그런데 자동차가 없어서 걸어다니거나 대중교통을 이용해야해.
그리고 여행 마지막 날은 오후 5시에 부산역에서 출발해.
여동생이랑 둘이서 가려고 하고, 예산은 50만원 내외로 부탁해."""

In [5]:
from IPython.display import display, Markdown

result = chain.invoke({"text": text})
display(Markdown(result))

<요약>
- 여행 일정 : 2박 3일, 내일 오전 8시 출발, 마지막 날 오후 5시 부산역 출발  
- 교통편 일정 : 서울역에서 부산역까지 오전 8시 출발, 오전 11시 도착, 부산 내에서는 대중교통 및 도보 이용  
- 여행 장소 : 부산 기장군 부근  
- 여행 스타일 : 맛집 탐방과 명소 방문, 도보 및 대중교통 이용  
- 예산 : 50만원 내외 (2인 기준)  
- 추천 숙소 : 기장군 내 대중교통 접근성이 좋은 게스트하우스 또는 중저가 호텔

<일정>
| 날짜       | 시간        | 장소/활동                      | 세부 계획 및 비고                                    |
|------------|-------------|-------------------------------|----------------------------------------------------|
| 1일차 (내일) | 08:00       | 서울역 출발 (KTX)              | KTX로 부산역까지 이동 (약 3시간 소요)              |
|            | 11:00       | 부산역 도착                   | 부산역 도착 후 기장군으로 이동 (부산 지하철 및 버스 이용) |
|            | 12:30       | 점심 식사 - 기장시장 내 맛집  | 기장시장 내 신선한 해산물 및 해물탕 추천              |
|            | 14:00       | 기장 죽성드림성당 방문        | 부산의 대표 명소, 사진 촬영 및 산책                  |
|            | 15:30       | 기장 해녀촌 방문              | 해녀 문화 체험 및 주변 카페에서 휴식                  |
|            | 17:00       | 숙소 체크인                   | 기장군 내 대중교통 접근성 좋은 게스트하우스 또는 중저가 호텔 |
|            | 18:30       | 저녁 식사 - 기장 대변항 횟집 | 신선한 회와 해산물 요리 맛보기                        |
|            | 20:00       | 숙소 복귀 및 휴식             | 숙소 주변 산책 가능                                   |
| 2일차       | 08:00       | 아침 식사 - 숙소 또는 근처 카페 | 가벼운 아침 식사                                    |
|            | 09:30       | 기장 오랑대 해변 산책         | 해안 절경 감상 및 사진 촬영                          |
|            | 11:00       | 기장 연화리 해수욕장 방문     | 해변 산책 및 주변 카페에서 휴식                       |
|            | 12:30       | 점심 식사 - 연화리 근처 맛집  | 해산물 또는 한식 전문점 추천                           |
|            | 14:00       | 아난티 코브 방문              | 리조트 내 산책 및 쇼핑, 카페 이용                      |
|            | 16:00       | 기장 죽성리 해안도로 드라이브 및 산책 | 대중교통 및 택시 이용 가능                            |
|            | 18:30       | 저녁 식사 - 기장군 내 유명 맛집 | 돼지국밥 또는 해물찜 추천                              |
|            | 20:00       | 숙소 복귀 및 휴식             | 숙소 주변에서 휴식 및 자유시간                         |
| 3일차       | 08:00       | 아침 식사 - 숙소 또는 근처 카페 | 가벼운 아침 식사                                    |
|            | 09:30       | 기장 대변항 산책 및 기념품 구매 | 신선한 해산물 시장 구경 및 기념품 구매                |
|            | 11:00       | 해동 용궁사 방문              | 부산 대표 해안 사찰 방문, 도보 및 대중교통 이용        |
|            | 13:00       | 점심 식사 - 해동 용궁사 근처 맛집 | 해산물 또는 한식 추천                                 |
|            | 14:30       | 부산역으로 이동               | 대중교통 이용 (버스 또는 지하철)                       |
|            | 17:00       | 부산역 출발 (KTX)             | 서울로 귀가                                          |

---

### 예산 참고 (2인 기준, 약 50만원 내외)
- KTX 왕복: 약 10만원 (2인)
- 숙박 (2박): 약 10~15만원 (게스트하우스 또는 중저가 호텔)
- 식사: 약 10~15만원 (맛집 위주, 6끼 기준)
- 교통비 및 기타: 약 5~10만원 (대중교통, 택시, 입장료 등)

---

### 추천 숙소 예시
- 기장군 내 ‘기장 게스트하우스’ (대중교통 접근성 우수, 1박 약 5~7만원)
- 중저가 호텔: ‘기장 베스트 호텔’ 또는 ‘기장 힐링 호텔’ 등

---

위 일정은 맛집 탐방과 명소 방문을 중심으로 도보 및 대중교통 이용에 최적화하여 동선을 구성하였습니다. 즐거운 여행 되세요!

## 4. Gradio Chat Interface

In [8]:
model = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.4,
    top_p=0.7
)
history_messages = []

In [ ]:
from langchain_core.prompts import MessagesPlaceholder

def get_summarize_chain(model):
    template = """
{text}

위에 입력된 텍스트를 다음 항목으로 요약해주세요: 
- 여행 일정 :
- 교통편 일정 :
- 여행 장소 :
- 여행 스타일 :
- 예산 :
- 추천 숙소 :"""

    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 여행 일정 작성을 도와주는 AI 어시스턴트입니다."),
        ("human", template)
    ])
    return prompt | model | StrOutputParser()

def get_planner_chain(model):
    template = """
다음 텍스트의 여행 일정을 기반으로 세부 여행 일정을 짜주세요.
텍스트: {summary}
규칙:
1. 날짜 및 시간과 장소, 세부 계획 항목으로 표 형태로 작성하세요.
2. 여행 스타일과 추천 숙소, 예산에 맞추어 동선을 고려하여 장소를 추천하세요.
답변:"""

    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder("chat_history"),
        ("human", template)
    ])
    return prompt | model | StrOutputParser()


In [ ]:

from langchain_core.messages import HumanMessage, AIMessage

def update_history(x):
    history_messages.append(HumanMessage(content=x["summary"]))
    history_messages.append(AIMessage(content=x["plan"]))
    return f"<요약>\n{x['summary']}\n\n<일정>\n{x['plan']}"

In [16]:
def answer_invoke(message, history):
    summarize_chain = get_summarize_chain(model)
    planner_chain = get_planner_chain(model)

    chain = (
        RunnableMap({
            "summary": summarize_chain,
            "chat_history": RunnableLambda(lambda _: history_messages)
         }) |
        RunnableMap({
            "summary": itemgetter("summary"),
            "plan": planner_chain
        }) |
        RunnableLambda(update_history)
    )
    response = chain.invoke({"text": message})
    return response

In [17]:
import gradio as gr

demo = gr.ChatInterface(
    fn=answer_invoke,         # 메시지 처리 함수
    title="여행 일정 어시스턴트", # 채팅 인터페이스의 제목
    type="messages"
)

# Gradio 인터페이스 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [18]:
demo.close()

Closing server running on port: 7860


In [19]:
history_messages

[HumanMessage(content='- 여행 일정 : 2박 3일 (첫날 오전 11시 부산역 도착, 마지막 날 오후 5시 부산역 출발)\n- 교통편 일정 : 서울역 → 부산역(오전 8시 출발, 오전 11시 도착) / 부산역 → 서울역(마지막 날 오후 5시 출발), 현지에서는 도보 및 대중교통 이용\n- 여행 장소 : 부산 기장군 부근\n- 여행 스타일 : 맛집 탐방 및 명소 방문, 도보 및 대중교통 위주, 여동생과 둘이 여행\n- 예산 : 50만원 내외\n- 추천 숙소 : 기장군 인근 대중교통 접근성 좋은 숙소(게스트하우스, 호텔 등)', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네, 요청하신 조건에 맞춰 부산 기장군 부근 2박 3일 여행 일정을 표로 정리해 드립니다.  \n(숙소는 기장역 또는 오시리아역 인근의 대중교통 접근성 좋은 호텔/게스트하우스 기준으로 추천합니다.)\n\n---\n\n| 날짜         | 시간           | 장소(주소)                              | 세부 계획                                                      |\n|--------------|----------------|----------------------------------------|--------------------------------------------------------------|\n| 1일차 (금)   | 11:00          | 부산역                                 | 부산역 도착, 대중교통(동해선) 이용해 기장 이동                |\n|              | 12:00          | 기장역/숙소 체크인                     | 숙소에 짐 보관 또는 얼리 체크인 (예: 호텔라온, 기장 게스트하우스 등) |\n|          